In [14]:
import pandas as pd
import numpy as np
import dynamic_portfolio.preprocess as prep
import dynamic_portfolio.utils as utils
import dynamic_portfolio.cross_validate as cv
# import dynamic_portfolio.models as model
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
import warnings
%load_ext autoreload
%autoreload 2
warnings.filterwarnings(action='ignore')
pd.set_option('display.max_columns', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
cross_val = {
    'fold_length' : 252, # Working days for 1 year
    'fold_stride' : 60, # Step between folds, here one quarter
    'train_test_ratio' : 0.7, # Split in fold
    'input_length' : 0, # Number of days to move back from last train_index, here 0
    'horizon' : 1, # Number of days ahead to make prediction, here 1
    'output_length' : 1, # Number of targets wanted
}
#Split the dataset by FOLDS
def get_folds(
    df: pd.DataFrame,
    fold_length = cross_val['fold_length'],
    fold_stride = cross_val['fold_stride']):
    '''
    This function slides through the Time Series dataframe of shape (n_timesteps, n_features) to create folds
    - of equal `fold_length`
    - using `fold_stride` between each fold
    Returns a list of folds, each as a DataFrame
    '''
    folds = []
    for idx in range(0, len(df), fold_stride):
        # Exits the loop as soon as the last fold index would exceed the last index
        if (idx + fold_length) > len(df):
            break
        fold = df.iloc[idx:idx + fold_length, :]
        folds.append(fold)
    return folds
#Split FOLDS by Train et Test
#### FOR ONE FOLD !!!!!
def train_test_split(fold: pd.DataFrame,
                     train_test_ratio = cross_val['train_test_ratio'],
                     input_length = cross_val['input_length']):
    '''
    Returns a train dataframe and a test dataframe (fold_train, fold_test)
    from which one can sample (X,y) sequences.
    df_train should contain all the timesteps until round(train_test_ratio * len(fold))
    '''
    # TRAIN SET
    # ======================
    last_train_idx = round(train_test_ratio * len(fold))
    fold_train = fold.iloc[0:last_train_idx, :]
    # TEST SET
    # ======================
    first_test_idx = last_train_idx - input_length # faut il pas changer le - en + ??
    fold_test = fold.iloc[first_test_idx:, :]
    return (fold_train, fold_test)
def cross_validate_ml(df, model) :
    '''
    get_folds() create many FOLDS, train_test_split() create a split on ONE FOLDS.
    The goal of this function is to make splits and sequences on each FOLDS.
    Then, apply a model.
    '''
    folds = get_folds(df, fold_length = cross_val['fold_length'], fold_stride = cross_val['fold_stride']) # 1 - Creating FOLDS
    scores =[]
    baseline = []
    for fold in folds:
        # 2 - CHRONOLOGICAL TRAIN TEST SPLIT of the current FOLD
        (fold_train, fold_test) = train_test_split(fold = fold,
                                                train_test_ratio = cross_val['train_test_ratio'],
                                                input_length = cross_val['input_length'] ,
                                                )
        # 3 - Scanninng fold_train and fold_test for SEQUENCES
        X_train, y_train = fold_train, fold_train['return']
        X_test, y_test = fold_test, fold_test['return']
        model = model
        model.fit(X_train, y_train)
        rmse_model = (mean_squared_error(y_test, model.predict(X_test)))**0.5
        scores.append(rmse_model)
        rmse_baseline = mean_squared_error(y_test.iloc[[0]], y_train.iloc[[-1]])**0.5
        baseline.append(rmse_baseline)
    return np.mean(scores), np.mean(baseline)

In [40]:
from sklearn.ensemble import RandomForestRegressor

# \
dict_score_for = {}
tickers = utils.return_tickers()
for ticker in tickers:
        dict_score_for [ticker] = cross_validate_ml(prep.ready_to_train_df(ticker), RandomForestRegressor(n_estimators = 100))
        print(f'done for ticker {ticker}')

done for ticker AAPL
done for ticker MSFT
done for ticker GOOG
done for ticker AMZN
done for ticker TSLA
done for ticker UNH
done for ticker XOM
done for ticker JNJ
done for ticker WMT
done for ticker NVDA
done for ticker JPM
done for ticker V
done for ticker CVX
done for ticker PG
done for ticker LLY
done for ticker MA
done for ticker HD
done for ticker META
done for ticker BAC
done for ticker ABBV
done for ticker PFE
done for ticker KO
done for ticker MRK
done for ticker PEP
done for ticker COST
done for ticker ORCL
done for ticker AVGO
done for ticker TMO
done for ticker MCD
done for ticker CSCO
done for ticker ACN
done for ticker DHR
done for ticker TMUS
done for ticker ABT
done for ticker WFC
done for ticker DIS
done for ticker LIN
done for ticker NEE
done for ticker BMY
done for ticker NKE
done for ticker VZ
done for ticker TXN
done for ticker UPS
done for ticker COP
done for ticker ADBE
done for ticker CMCSA
done for ticker CRM
done for ticker PM
done for ticker MS
done for tick

done for ticker JKHY
done for ticker MGM
done for ticker ESS
done for ticker L
done for ticker PEAK
done for ticker MTCH
done for ticker NDSN
done for ticker EVRG
done for ticker IPG
done for ticker BEN
done for ticker CBOE
done for ticker TECH
done for ticker SIVB
done for ticker VFC
done for ticker IP
done for ticker HST
done for ticker UDR
done for ticker POOL
done for ticker RE
done for ticker PARA
done for ticker SNA
done for ticker CPT
done for ticker LW
done for ticker PKG
done for ticker CRL
done for ticker SWK
done for ticker BIO
done for ticker WDC
done for ticker CHRW
done for ticker STX
done for ticker MAS
done for ticker GL
done for ticker CE
done for ticker REG
done for ticker NI
done for ticker BXP
done for ticker HSIC
done for ticker CCL
done for ticker TFX
done for ticker CZR
done for ticker NWS
done for ticker NWSA
done for ticker KMX
done for ticker EMN
done for ticker JNPR
done for ticker PHM
done for ticker CDAY
done for ticker ALLE
done for ticker QRVO
done for ti

In [37]:
dict_score

{'AAPL': (0.003516573395309949, 0.02199600990057482),
 'MSFT': (0.0036971220034474842, 0.016999351555133206),
 'GOOG': (0.004168121807827624, 0.01786548530030592),
 'AMZN': (0.007086589818893358, 0.02599608703713922),
 'TSLA': (0.0074855688525229435, 0.03814787938554345),
 'UNH': (0.0035592407503447815, 0.015673685442134145),
 'XOM': (0.0026633919678586194, 0.017162719947628968),
 'JNJ': (0.002333278323128976, 0.014323481132729866),
 'WMT': (0.0024300008384675687, 0.011165560059919543),
 'NVDA': (0.007151395817790712, 0.029186713053666342),
 'JPM': (0.00395524938293938, 0.021057143344779167),
 'V': (0.0031299972142692645, 0.017945043356423062),
 'CVX': (0.002605285719065406, 0.017422256704201933),
 'PG': (0.002050358180677263, 0.012557542143500027),
 'LLY': (0.0026551190587628224, 0.014675241210098954),
 'MA': (0.003953315524094866, 0.019448614539125487),
 'HD': (0.00248195785235346, 0.017586914924342258),
 'META': (0.005277341779233381, 0.020825786500747085),
 'BAC': (0.00493113368736

In [39]:
rmse= []
baseline = []
for key in dict_score.keys():
    rmse.append(dict_score[key][0])
    baseline.append(dict_score[key][1])
print(np.mean(rmse), np.mean(baseline) )

0.004374310270270182 0.02096529727516623


In [41]:
rmse= []
baseline = []
for key in dict_score_for.keys():
    rmse.append(dict_score_for[key][0])
    baseline.append(dict_score_for[key][1])
print(np.mean(rmse), np.mean(baseline) )

0.004353121672543864 0.02096529727516623


In [10]:
import pandas as pd
import numpy as np
import dynamic_portfolio.preprocess as prep
import dynamic_portfolio.utils as utils
import dynamic_portfolio.cross_validate as cv
import dynamic_portfolio.models as model
from sklearn.ensemble import GradientBoostingRegressor
import warnings
%load_ext autoreload
%autoreload 2
warnings.filterwarnings(action='ignore')
pd.set_option('display.max_columns', None)



In [24]:
rmse = []
baseline = []
params = []

def custom_gridsearch(df, model, max_depth=[2, 5, 8], criterion = ['squared_error', 'absolute_error'], n_estimator=[100,200,500]):
    counter = 0
    for max_depth_i in max_depth:
        for criterion_i in criterion:
            for n_estimator_i in n_estimator:
#                for learning_rate_i in learning_rate:
                test = cross_validate_ml(df = df, model = model(max_depth=max_depth_i,
                                                                   criterion = criterion_i,
                                                                   n_estimators = n_estimator_i,
#                                                                   learning_rate = learning_rate_i
                                                                   ))
                rmse.append(test[0])
                baseline.append(test[1])
                params.append((max_depth_i, criterion_i, n_estimator_i))
                counter += 1
                print(f'model {counter} done with parameters {max_depth_i}, {criterion_i}, {n_estimator_i}, rmse = {test[0]}')
    idx_min = np.argmin(rmse)
    best_params = params[idx_min]
    
    return best_params

In [25]:
from sklearn.ensemble import RandomForestRegressor

grid_score = custom_gridsearch(prep.ready_to_train_df('IBM'), model = RandomForestRegressor)

model 1 done with parameters 2, squared_error, 100, rmse = 0.005102034938267749
model 2 done with parameters 2, squared_error, 200, rmse = 0.005079307603735572
model 3 done with parameters 2, squared_error, 500, rmse = 0.005057381252707621
model 4 done with parameters 2, absolute_error, 100, rmse = 0.005879748078525165
model 5 done with parameters 2, absolute_error, 200, rmse = 0.005891330623601229
model 6 done with parameters 2, absolute_error, 500, rmse = 0.005892385844292436
model 7 done with parameters 5, squared_error, 100, rmse = 0.0030118426226079286
model 8 done with parameters 5, squared_error, 200, rmse = 0.0030089622562180017
model 9 done with parameters 5, squared_error, 500, rmse = 0.002973763609355252
model 10 done with parameters 5, absolute_error, 100, rmse = 0.002992421501915195
model 11 done with parameters 5, absolute_error, 200, rmse = 0.0029572319933501437
model 12 done with parameters 5, absolute_error, 500, rmse = 0.0029710639594408587
model 13 done with paramete

In [31]:
from sklearn.decomposition import PCA

pca_dict_score = {}
tickers = ['IBM']
#utils.return_tickers()
for ticker in tickers:
    pca = PCA(n_components=0.9)
    ticker_df = prep.ready_to_train_df(ticker)
    pca.fit(ticker_df)
    n_pcs= pca.n_components_
    most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]
    initial_feature_names = ticker_df.columns
    most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]
    pca_df = ticker_df[list(np.unique(most_important_names))] 
    returns = ticker_df[['return']]
    final_pca = pd.merge(pca_df, returns, how='outer', left_index=True, right_index=True)
    pca_dict_score[ticker] = cross_validate_ml(final_pca, RandomForestRegressor(max_depth = 5, 
                                                                                                         criterion='absolute_error',
                                                                                                       # learning_rate=0.1,
                                                                                                        n_estimators = 200))
    print(f"done for ticker {ticker} index # {tickers.index(ticker)}")

done for ticker IBM index # 0


In [32]:
pca_dict_score

{'IBM': (0.0026797440209805103, 0.014803027940034318)}

In [35]:
from sklearn.linear_model import Ridge

pca_dict_score = {}
tickers = utils.return_tickers()
for ticker in tickers:
    pca = PCA(n_components=0.9)
    ticker_df = prep.ready_to_train_df(ticker)
    pca.fit(ticker_df)
    n_pcs= pca.n_components_
    most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]
    initial_feature_names = ticker_df.columns
    most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]
    pca_df = ticker_df[list(np.unique(most_important_names))] 
    returns = ticker_df[['return']]
    final_pca = pd.merge(pca_df, returns, how='outer', left_index=True, right_index=True)
    pca_dict_score[ticker] = cross_validate_ml(final_pca, Ridge())
        #max_depth = 5, 
                                                                                                         #criterion='absolute_error',
                                                                                                       # learning_rate=0.1,
                                                                                                       # n_estimators = 200)
    
    print(f"done for ticker {ticker} index # {tickers.index(ticker)}")

done for ticker AAPL index # 0
done for ticker MSFT index # 1
done for ticker GOOG index # 2
done for ticker AMZN index # 3
done for ticker TSLA index # 4
done for ticker UNH index # 5
done for ticker XOM index # 6
done for ticker JNJ index # 7
done for ticker WMT index # 8
done for ticker NVDA index # 9
done for ticker JPM index # 10
done for ticker V index # 11
done for ticker CVX index # 12
done for ticker PG index # 13
done for ticker LLY index # 14
done for ticker MA index # 15
done for ticker HD index # 16
done for ticker META index # 17
done for ticker BAC index # 18
done for ticker ABBV index # 19
done for ticker PFE index # 20
done for ticker KO index # 21
done for ticker MRK index # 22
done for ticker PEP index # 23
done for ticker COST index # 24
done for ticker ORCL index # 25
done for ticker AVGO index # 26
done for ticker TMO index # 27
done for ticker MCD index # 28
done for ticker CSCO index # 29
done for ticker ACN index # 30
done for ticker DHR index # 31
done for tic

done for ticker BKR index # 259
done for ticker GLW index # 260
done for ticker LYB index # 261
done for ticker ES index # 262
done for ticker BAX index # 263
done for ticker STT index # 264
done for ticker VRSK index # 265
done for ticker TROW index # 266
done for ticker WBD index # 267
done for ticker AWK index # 268
done for ticker IT index # 269
done for ticker GPN index # 270
done for ticker HRL index # 271
done for ticker FANG index # 272
done for ticker WTW index # 273
done for ticker RJF index # 274
done for ticker GPC index # 275
done for ticker IFF index # 276
done for ticker CDW index # 277
done for ticker TSCO index # 278
done for ticker FITB index # 279
done for ticker ARE index # 280
done for ticker URI index # 281
done for ticker ZBH index # 282
done for ticker K index # 283
done for ticker LEN index # 284
done for ticker EBAY index # 285
done for ticker EIX index # 286
done for ticker CBRE index # 287
done for ticker EFX index # 288
done for ticker VMC index # 289
done 

In [36]:
pca_dict_score

{'AAPL': (0.02313130296312449, 0.02199600990057482),
 'MSFT': (0.0190224404816197, 0.016999351555133206),
 'GOOG': (0.018241800958080718, 0.01786548530030592),
 'AMZN': (0.02788706754317082, 0.02599608703713922),
 'TSLA': (0.030494441410021035, 0.03814787938554345),
 'UNH': (0.01863629587826702, 0.015673685442134145),
 'XOM': (0.013376565831141052, 0.017162719947628968),
 'JNJ': (0.011416929456688726, 0.014323481132729866),
 'WMT': (0.01393573835742472, 0.011165560059919543),
 'NVDA': (0.03038501280527252, 0.029186713053666342),
 'JPM': (0.02331168431553059, 0.021057143344779167),
 'V': (0.015604923515285541, 0.017945043356423062),
 'CVX': (0.01396621079938117, 0.017422256704201933),
 'PG': (0.010809132214197053, 0.012557542143500027),
 'LLY': (0.015117153687441036, 0.014675241210098954),
 'MA': (0.020489870588639873, 0.019448614539125487),
 'HD': (0.016122826492745525, 0.017586914924342258),
 'META': (0.02423810643186391, 0.020825786500747085),
 'BAC': (0.020303527708102048, 0.0209982

In [37]:
rmse= []
baseline = []
for key in pca_dict_score.keys():
    rmse.append(pca_dict_score[key][0])
    baseline.append(pca_dict_score[key][1])
print(np.mean(rmse), np.mean(baseline) )

0.021315474725177704 0.02096529727516623


In [42]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso


dict_score = {}
tickers = utils.return_tickers()
for ticker in tickers:
        dict_score [ticker] = cross_validate_ml(prep.ready_to_train_df(ticker), StackingRegressor(estimators=[('Ridge',Ridge()),('L',Lasso())], final_estimator=LinearRegression()))
        print(f'done for ticker {ticker}')

done for ticker AAPL
done for ticker MSFT
done for ticker GOOG
done for ticker AMZN
done for ticker TSLA
done for ticker UNH
done for ticker XOM
done for ticker JNJ
done for ticker WMT
done for ticker NVDA
done for ticker JPM
done for ticker V
done for ticker CVX
done for ticker PG
done for ticker LLY
done for ticker MA
done for ticker HD
done for ticker META
done for ticker BAC
done for ticker ABBV
done for ticker PFE
done for ticker KO
done for ticker MRK
done for ticker PEP
done for ticker COST
done for ticker ORCL
done for ticker AVGO
done for ticker TMO
done for ticker MCD
done for ticker CSCO
done for ticker ACN
done for ticker DHR
done for ticker TMUS
done for ticker ABT
done for ticker WFC
done for ticker DIS
done for ticker LIN
done for ticker NEE
done for ticker BMY
done for ticker NKE
done for ticker VZ
done for ticker TXN
done for ticker UPS
done for ticker COP
done for ticker ADBE
done for ticker CMCSA
done for ticker CRM
done for ticker PM
done for ticker MS
done for tick

done for ticker JKHY
done for ticker MGM
done for ticker ESS
done for ticker L
done for ticker PEAK
done for ticker MTCH
done for ticker NDSN
done for ticker EVRG
done for ticker IPG
done for ticker BEN
done for ticker CBOE
done for ticker TECH
done for ticker SIVB
done for ticker VFC
done for ticker IP
done for ticker HST
done for ticker UDR
done for ticker POOL
done for ticker RE
done for ticker PARA
done for ticker SNA
done for ticker CPT
done for ticker LW
done for ticker PKG
done for ticker CRL
done for ticker SWK
done for ticker BIO
done for ticker WDC
done for ticker CHRW
done for ticker STX
done for ticker MAS
done for ticker GL
done for ticker CE
done for ticker REG
done for ticker NI
done for ticker BXP
done for ticker HSIC
done for ticker CCL
done for ticker TFX
done for ticker CZR
done for ticker NWS
done for ticker NWSA
done for ticker KMX
done for ticker EMN
done for ticker JNPR
done for ticker PHM
done for ticker CDAY
done for ticker ALLE
done for ticker QRVO
done for ti

In [44]:
rmse= []
baseline = []
for key in dict_score.keys():
    rmse.append(dict_score[key][0])
    baseline.append(dict_score[key][1])
print(np.mean(rmse), np.mean(baseline) )

0.02068358877182557 0.02096529727516623
